### RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [20]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [21]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 3 PDF files to process

Processing: 1804.06812v1.pdf
  ✓ Loaded 22 pages

Processing: ECG_Arrhythmia_Classification_Using_STFT-Based_Spectrogram_and_Convolutional_Neural_Network.pdf
  ✓ Loaded 10 pages

Processing: fphys-14-1246746.pdf
  ✓ Loaded 20 pages

Total documents loaded: 52


In [22]:
all_pdf_documents

[Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-04-19T00:44:32+00:00', 'author': '', 'keywords': '', 'moddate': '2018-04-19T00:44:32+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdfs\\1804.06812v1.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1', 'source_file': '1804.06812v1.pdf', 'file_type': 'pdf'}, page_content='Noname manuscript No.\n(will be inserted by the editor)\nECG arrhythmia classiﬁcation using a 2-D\nconvolutional neural network\nT ae Joon Jun 1 · Hoang Minh Nguyen 1 ·\nDaeyoun Kang 1 · Dohyeun Kim 1 ·\nDaeyoung Kim 1 · Y oung-Hak Kim2\nReceived: date / Accepted: date\nAbstract In this paper, we propose an eﬀective electrocardiogram (ECG)\narrhythmia classiﬁcation method using a deep two-dimensional convolutional\nneural network (CNN) which recently shows outstandin

In [23]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [24]:
chunks=split_documents(all_pdf_documents)
chunks

Split 52 documents into 266 chunks

Example chunk:
Content: Noname manuscript No.
(will be inserted by the editor)
ECG arrhythmia classiﬁcation using a 2-D
convolutional neural network
T ae Joon Jun 1 · Hoang Minh Nguyen 1 ·
Daeyoun Kang 1 · Dohyeun Kim 1 ·
Da...
Metadata: {'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-04-19T00:44:32+00:00', 'author': '', 'keywords': '', 'moddate': '2018-04-19T00:44:32+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdfs\\1804.06812v1.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1', 'source_file': '1804.06812v1.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-04-19T00:44:32+00:00', 'author': '', 'keywords': '', 'moddate': '2018-04-19T00:44:32+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdfs\\1804.06812v1.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1', 'source_file': '1804.06812v1.pdf', 'file_type': 'pdf'}, page_content='Noname manuscript No.\n(will be inserted by the editor)\nECG arrhythmia classiﬁcation using a 2-D\nconvolutional neural network\nT ae Joon Jun 1 · Hoang Minh Nguyen 1 ·\nDaeyoun Kang 1 · Dohyeun Kim 1 ·\nDaeyoung Kim 1 · Y oung-Hak Kim2\nReceived: date / Accepted: date\nAbstract In this paper, we propose an eﬀective electrocardiogram (ECG)\narrhythmia classiﬁcation method using a deep two-dimensional convolutional\nneural network (CNN) which recently shows outstandin

### embedding And vectorStoreDB

In [25]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 473.73it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded successfully. Embedding dimension: 384


### VectorStore

In [27]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 1862


In [28]:
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-04-19T00:44:32+00:00', 'author': '', 'keywords': '', 'moddate': '2018-04-19T00:44:32+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdfs\\1804.06812v1.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1', 'source_file': '1804.06812v1.pdf', 'file_type': 'pdf'}, page_content='Noname manuscript No.\n(will be inserted by the editor)\nECG arrhythmia classiﬁcation using a 2-D\nconvolutional neural network\nT ae Joon Jun 1 · Hoang Minh Nguyen 1 ·\nDaeyoun Kang 1 · Dohyeun Kim 1 ·\nDaeyoung Kim 1 · Y oung-Hak Kim2\nReceived: date / Accepted: date\nAbstract In this paper, we propose an eﬀective electrocardiogram (ECG)\narrhythmia classiﬁcation method using a deep two-dimensional convolutional\nneural network (CNN) which recently shows outstandin

In [29]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 266 texts...


Batches: 100%|██████████| 9/9 [00:01<00:00,  6.94it/s]


Generated embeddings with shape: (266, 384)
Adding 266 documents to vector store...
Successfully added 266 documents to vector store
Total documents in collection: 2128


### Retriever Pipeline From VectorStore

In [30]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)



In [31]:
rag_retriever

In [32]:
rag_retriever.retrieve("What are arrhythmias")

Retrieving documents for query: 'What are arrhythmias'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 49.92it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_7dd180c0_172',
  'content': 'myocardial ischemia or damage.\nA thorough comprehension of these components, intervals,\nand segments is required to interpret ECG results accurately.\nAnalyzing their properties, variations, and correlations helps to\ndiagnose various heart diseases and guides proper management and\ntreatment techniques. Arrhythmias can be asymptomatic or severe,\ncausing palpitations, dizziness, chest pain, shortness of breath, and\npossibly loss of consciousness ( Abbott, 2005 ). Arrhythmias are\nfrequently diagnosed and classified by examining ECG records,\nwhich give vital information on the heart’ s electrical activity.\nDepending on the nature and degree of the arrhythmia, treatment\noptions may include medication, electrical cardioversion, catheter\nablation, or implanted devices such as pacemakers or implantable\ncardioverter-defibrillators (ICDs). A thorough examination and\nprecise categorization of arrhythmias are required to develop\nsuitable trea

### RAG Pipeline- VectorDB To LLM Output Generation

### Using Grok LLM

In [ ]:
# import os
# from dotenv import load_dotenv
# load_dotenv()

# print(os.getenv("GROQ_API_KEY"))

None


In [ ]:
# from langchain_groq import ChatGroq
# from langchain.prompts import PromptTemplate
# from langchain.schema import HumanMessage, SystemMessage

ModuleNotFoundError: No module named 'langchain.prompts'

In [ ]:
# class GroqLLM:
#     def __init__(self, model_name: str = "gemma2-9b-it", api_key: str =None):
#         """
#         Initialize Groq LLM
        
#         Args:
#             model_name: Groq model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
#             api_key: Groq API key (or set GROQ_API_KEY environment variable)
#         """
#         self.model_name = model_name
#         self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        
#         if not self.api_key:
#             raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter.")
        
#         self.llm = ChatGroq(
#             groq_api_key=self.api_key,
#             model_name=self.model_name,
#             temperature=0.1,
#             max_tokens=1024
#         )
        
#         print(f"Initialized Groq LLM with model: {self.model_name}")

#     def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
#         """
#         Generate response using retrieved context
        
#         Args:
#             query: User question
#             context: Retrieved document context
#             max_length: Maximum response length
            
#         Returns:
#             Generated response string
#         """
        
#         # Create prompt template
#         prompt_template = PromptTemplate(
#             input_variables=["context", "question"],
#             template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

# Context:
# {context}

# Question: {question}

# Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
#         )
        
#         # Format the prompt
#         formatted_prompt = prompt_template.format(context=context, question=query)
        
#         try:
#             # Generate response
#             messages = [HumanMessage(content=formatted_prompt)]
#             response = self.llm.invoke(messages)
#             return response.content
            
#         except Exception as e:
#             return f"Error generating response: {str(e)}"
        
#     def generate_response_simple(self, query: str, context: str) -> str:
#         """
#         Simple response generation without complex prompting
        
#         Args:
#             query: User question
#             context: Retrieved context
            
#         Returns:
#             Generated response
#         """
#         simple_prompt = f"""Based on this context: {context}

# Question: {query}

# Answer:"""
        
#         try:
#             messages = [HumanMessage(content=simple_prompt)]
#             response = self.llm.invoke(messages)
#             return response.content
#         except Exception as e:
#             return f"Error: {str(e)}"
    


### Connecting HuggingFace model to Langchain

In [33]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_community.llms import HuggingFacePipeline


class HuggingFaceLocalLLM:
    def __init__(
        self,
        # model_name: str = "microsoft/Phi-3-mini-4k-instruct",
        model_name: str = "google/gemma-2-2b-it",
        temperature: float = 0.1,
        max_tokens: int = 1024,
    ):
        """
        Initialize a local Hugging Face LLM (GPU)
        """
        self.model_name = model_name

        print(f"Loading local HF model: {model_name}")

        tokenizer = AutoTokenizer.from_pretrained(model_name)

        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            # device_map="auto",          # auto GPU placement
            torch_dtype=torch.float16,  # saves VRAM
            trust_remote_code=True
        )

        model = model.to("cuda")

        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
        )

        self.llm = HuggingFacePipeline(pipeline=pipe)

        print("Local Hugging Face LLM initialized successfully")

    def invoke(self, prompt: str) -> str:
        """
        Match the GroqLLM interface.
        Accepts a single string prompt and returns the generated text.
        """
        return self.llm.invoke(prompt)

    def generate_response(self, query: str, context: str) -> str:
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer:"""
        )

        formatted_prompt = prompt_template.format(
            context=context,
            question=query
        )

        try:
            return self.llm.invoke(formatted_prompt)
        except Exception as e:
            return f"Error generating response: {str(e)}"

    def generate_response_simple(self, query: str, context: str) -> str:
        simple_prompt = f"""Based on this context:

{context}

Question: {query}

Answer:"""

        try:
            return self.llm.invoke(simple_prompt)
        except Exception as e:
            return f"Error: {str(e)}"




In [34]:
# Initialize Groq LLM (you'll need to set GROQ_API_KEY environment variable)
try:
    hugg_llm = HuggingFaceLocalLLM()
    print("HuggingFaceLocalLLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please initialize HuggingFaceModel environment correctly.")
    hugg_llm = None

Loading local HF model: google/gemma-2-2b-it


Loading weights: 100%|██████████| 288/288 [00:08<00:00, 32.52it/s, Materializing param=model.norm.weight]                                


Local Hugging Face LLM initialized successfully
HuggingFaceLocalLLM initialized successfully!


In [35]:
### get the context from the retriever and pass it to the LLM

rag_retriever.retrieve("ECG arrhythmia")

Retrieving documents for query: 'ECG arrhythmia'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.46it/s]

Generated embeddings with shape: (1, 384)


Retrieved 5 documents (after filtering)


[{'id': 'doc_aff4c1c1_172',
  'content': 'myocardial ischemia or damage.\nA thorough comprehension of these components, intervals,\nand segments is required to interpret ECG results accurately.\nAnalyzing their properties, variations, and correlations helps to\ndiagnose various heart diseases and guides proper management and\ntreatment techniques. Arrhythmias can be asymptomatic or severe,\ncausing palpitations, dizziness, chest pain, shortness of breath, and\npossibly loss of consciousness ( Abbott, 2005 ). Arrhythmias are\nfrequently diagnosed and classified by examining ECG records,\nwhich give vital information on the heart’ s electrical activity.\nDepending on the nature and degree of the arrhythmia, treatment\noptions may include medication, electrical cardioversion, catheter\nablation, or implanted devices such as pacemakers or implantable\ncardioverter-defibrillators (ICDs). A thorough examination and\nprecise categorization of arrhythmias are required to develop\nsuitable trea

### Integration Vectordb Context pipeline With LLM output

In [36]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2", device=0)


def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 425.58it/s, Materializing param=roberta.encoder.layer.11.output.dense.weight]              
RobertaForQuestionAnswering LOAD REPORT from: deepset/roberta-base-squad2
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [37]:
answer=rag_simple("What is arrhythmia?",rag_retriever,llm=hugg_llm)
# answer = hugg_llm.generate_response_simple("What is arrhythmia?", rag_retriever)
print(answer)

Retrieving documents for query: 'What is arrhythmia?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 68.36it/s]
Both `max_new_tokens` (=1024) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Human: Use the following context to answer the question concisely.
        Context:
        myocardial ischemia or damage.
A thorough comprehension of these components, intervals,
and segments is required to interpret ECG results accurately.
Analyzing their properties, variations, and correlations helps to
diagnose various heart diseases and guides proper management and
treatment techniques. Arrhythmias can be asymptomatic or severe,
causing palpitations, dizziness, chest pain, shortness of breath, and
possibly loss of consciousness ( Abbott, 2005 ). Arrhythmias are
frequently diagnosed and classified by examining ECG records,
which give vital information on the heart’ s electrical activity.
Depending on the nature and degree of the arrhythmia, treatment
options may include medication, electrical cardioversion, catheter
ablation, or implanted devices such as pacemakers or implantable
cardioverter-defibril

### Enhanced RAG Pipeline Features

In [39]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("What is arrhythmia?", rag_retriever, hugg_llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'What is arrhythmia?'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 80.87it/s]
Both `max_new_tokens` (=1024) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Answer: Human: Use the following context to answer the question concisely.
Context:
myocardial ischemia or damage.
A thorough comprehension of these components, intervals,
and segments is required to interpret ECG results accurately.
Analyzing their properties, variations, and correlations helps to
diagnose various heart diseases and guides proper management and
treatment techniques. Arrhythmias can be asymptomatic or severe,
causing palpitations, dizziness, chest pain, shortness of breath, and
possibly loss of consciousness ( Abbott, 2005 ). Arrhythmias are
frequently diagnosed and classified by examining ECG records,
which give vital information on the heart’ s electrical activity.
Depending on the nature and degree of the arrhythmia, treatment
options may include medication, electrical cardioversion, catheter
ablation, or implanted devices such as pacemakers or implantable
cardioverter-defibrillators (